# MorphCT Example Workflow

1. Start with an atomistic snapshot
2. Determine which atom indices belong to which chromophore using [SMARTS](https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html) matching
3. Calculate the energies for each chromophore and chromophore pair using quantum chemical calculations (QCC)
4. Run the kinetic monte carlo (KMC) algorithm to calculate charge mobility

First let's import necessary modules and define a couple of useful functions for visualization:

In [ ]:
from copy import deepcopy
import os
import multiprocessing as mp

import gsd.hoomd
import mbuild as mb
import numpy as np

from morphct import execute_qcc as eqcc
from morphct import mobility_kmc as kmc
from morphct import chromophores
from morphct import kmc_analyze
from morphct.chromophores import conversion_dict


def visualize_qcc_input(qcc_input):
    """
    Visualize a quantum chemical input string (for pyscf) using mbuild.
    
    Parameters
    ----------
    qcc_input : str
        Input string to visualize
    """
    comp = mb.Compound()
    for line in qcc_input.split(";")[:-1]:
        atom, x, y, z = line.split()
        xyz = np.array([x,y,z], dtype=float)
        # Angstrom -> nm
        xyz /= 10
        comp.add(mb.Particle(name=atom,pos=xyz))
    comp.visualize().show()
    
def from_snapshot(snapshot, scale=1.0):
    """
    Convert a hoomd.data.Snapshot or a gsd.hoomd.Snapshot to an
    mbuild Compound.
    
    Parameters
    ----------
    snapshot : hoomd.data.SnapshotParticleData or gsd.hoomd.Snapshot
        Snapshot from which to build the mbuild Compound.
    scale : float, optional, default 1.0
        Value by which to scale the length values
        
    Returns
    -------
    comp : mb.Compound
    """
    comp = mb.Compound()
    bond_array = snapshot.bonds.group
    n_atoms = snapshot.particles.N

    # There will be a better way to do this once box overhaul merged
    try:
        # gsd
        box = snapshot.configuration.box
        comp.box = mb.box.Box(lengths=box[:3] * scale)
    except AttributeError:
        # hoomd
        box = snapshot.box
        comp.box = mb.box.Box(lengths=np.array([box.Lx,box.Ly,box.Lz]) * scale)

    # to_hoomdsnapshot shifts the coords, this will keep consistent
    shift = np.array(comp.box.lengths)/2
    # Add particles
    for i in range(n_atoms):
        name = snapshot.particles.types[snapshot.particles.typeid[i]]
        xyz = snapshot.particles.position[i] * scale + shift
        charge = snapshot.particles.charge[i]

        atom = mb.Particle(name=name, pos=xyz, charge=charge)
        comp.add(atom, label=str(i))

    # Add bonds
    particle_dict = {idx: p for idx, p in enumerate(comp.particles())}
    for i in range(bond_array.shape[0]):
        atom1 = int(bond_array[i][0])
        atom2 = int(bond_array[i][1])
        comp.add_bond([particle_dict[atom1], particle_dict[atom2]])
    return comp

Here's our starting structure, an atomistic (not coarse-grain or united atom) gsd file with 2 p3ht 15-mers:

In [ ]:
with gsd.hoomd.open(name='morphct/tests/assets/p3ht_2_15mers.gsd', mode='rb') as f:
    snap = f[0]
    
box = snap.configuration.box[:3]
unwrapped_positions = snap.particles.position + snap.particles.image * box

unwrap_snap = deepcopy(snap)
unwrap_snap.particles.position = unwrapped_positions
unwrap_snap.particles.types = [conversion_dict[i].symbol for i in snap.particles.types]
comp = from_snapshot(unwrap_snap, scale=0.1)
comp.visualize().show()

Next let's use SMARTS matching to detect our chromophores. This SMARTS string is for (a generalized) p3ht. The `conversion_dict` is a dictionary which converts atom type to element.

Note: The positions/orientations in the gsd file are not optimal, so openbabel has a tough time recognizing them as aromatic -- this is why I am defining the SMARTS by element (`[#6]`) instead of aromatic carbon (`c`) and even so, one chromophore is not detected correctly. If we were running a simulation workflow from scratch, I would recommend using the first frame (before any distortion) for smarts matching and then mapping those indices to the final structure.

In [ ]:
smarts_str = "[#6]1[#6][#16][#6][#6]1CCCCCC"

aaids = chromophores.get_chromo_ids_smiles(snap, smarts_str, conversion_dict)

For two 15-mers with each monomer being a chromophore, we expect 30 chromophores. Basically, the smarts matching misses one chromophore. So I have to add it manually. 

The visualization below shows the detected chromophores in pink and the missed one in blue:

In [ ]:
missed_inds = np.array([727,728,729,730,731,732,733,734,735,736,737])

for i,p in enumerate(comp.particles()):
    if i in np.hstack(aaids):
        p.name = "Kr"
    elif i in missed_inds:
        p.name = "N"
comp.visualize().show()

aaids.append(missed_inds)

Next let's make a Chromophore object for each detected chromophore and add them to a list:

In [ ]:
chromo_list = []
for i,aaid in enumerate(aaids):
    chromo_list.append(chromophores.Chromophore(i, snap, aaid, "donor", conversion_dict))

Next let's compute (using voronoi analysis) which chromophores are neighbors. `qcc_pairs` is a list containing the indices of the pair and the pair's qcc input: `((i,j), qcc_input)`


In [ ]:
qcc_pairs = chromophores.set_neighbors_voronoi(chromo_list, snap, conversion_dict, d_cut=min(box)/2)
print(f"There are {len(qcc_pairs)} chromophore pairs")

Before running any QCC we can check that the pair and singles inputs look reasonable. There won't be any bonds and hydrogen atoms should've been added.

In [ ]:
i = 13 # try any number from 0 to 180
print(f"Pair #{i}:")
visualize_qcc_input(qcc_pairs[i][1])

i = 0 # try any number from 0 to 29
print(f"Single #{i}:")
visualize_qcc_input(chromo_list[i].qcc_input)

Next we need to get the single and pair energies which will be used in the QCC calculations. These take a little time, so these functions are edesigned to save to a file. We'll first make the directory and define the filenames.

In [ ]:
outpath = os.path.join(os.getcwd(), "output")
if not os.path.exists(outpath):
    os.makedirs(outpath)
s_filename = os.path.join(outpath, "singles_energies.txt")
d_filename = os.path.join(outpath, "dimer_energies.txt")

MorphCT uses multiprocessing to run calculations in parallel.

`eqcc.get_homolumo(chromo_list[0].qcc_input)` operates on one chromophore and returns HOMO-1, HOMO, LUMO, LUMO+1

`eqcc.singles_homolumo` does this for all chromophores and saves the energies to a file.

In [ ]:
%%time
data = eqcc.singles_homolumo(chromo_list, s_filename)

# Approx time required:
# CPU times: user 20.9 ms, sys: 30.1 ms, total: 51 ms
# Wall time: 6.05 s

Next let's compute the pair energies:

In [ ]:
%%time
dimer_data = eqcc.dimer_homolumo(qcc_pairs, d_filename)

# Approx time required:
# CPU times: user 20.3 ms, sys: 27.2 ms, total: 47.6 ms
# Wall time: 56.6 s

Once the energy files are finsihed, we can use them to set the energy values of the chromophores in the list

In [ ]:
eqcc.set_energyvalues(chromo_list, s_filename, d_filename)

This function sets the homo_1, homo, lumo, lumo_1, neighbors_delta_e, and neighbors_ti of each chromphore.

In [ ]:
i = 0
chromo = chromo_list[i]
print(f"Chromophore {i}:")
print(f"HOMO-1: {chromo.homo_1:.2f} HOMO: {chromo.homo:.2f} LUMO: {chromo.lumo:.2f} LUMO+1: {chromo.lumo_1:.2f}")
print(f"{len(chromo.neighbors)} neighbors")
print(f"DeltaE of first neighbor: {chromo.neighbors_delta_e[0]:.3f}")
print(f"Transfer integral of first neighbor: {chromo.neighbors_ti[0]:.3f}")

With all the energy values set, we're ready to run KMC! First we'll make a directory and set a random seed to keep our results consistent:

In [ ]:
kmc_dir = os.path.join(outpath, "kmc")
if not os.path.exists(kmc_dir):
    os.makedirs(kmc_dir)
    
seed = 42

Next we'll create a random list of jobs:

In [ ]:
lifetimes = [1.00e-13, 1.00e-12]
jobs_list = kmc.get_jobslist(lifetimes, n_holes=10, seed=seed)

And we'll run these jobs in parallel using multiprocessing:

In [ ]:
temp = 300
combined_data = kmc.run_kmc(jobs_list, kmc_dir, chromo_list, snap, temp, verbose=1)

The output files for each process are saved in output/kmc/kmc_**proc#**.log:

In [ ]:
with open(os.path.join(kmc_dir, "kmc_00.log"), "r") as f:
    lines = f.readlines()
print(*lines)

Finally we can analyze our results -- plots will be saved in output/kmc/figures/

In [ ]:
kmc_analyze.main(combined_data, temp, chromo_list, snap, kmc_dir)